In [4]:
import numpy as np
import os

from pathlib import Path

# change working directory to make src visible
os.chdir(Path.cwd().parent)

from src.data_generation import SPDEventGenerator
from src.visualization import draw_event

In [5]:
event_gen = SPDEventGenerator(detector_eff=0.98)

for _ in range(2):
    event = event_gen.generate_spd_event()
    print(event)

Event:
Shape of real hits: (220, 3)
Shape of momentums: (220, 3)
Shape of fake hits: (1338, 3)
Fraction of fakes: 0.86
Fraction of missing hits: 0.02
Number of unique tracks: 7
Vertex: Vertex(x=12.82, y=-0.80, z=-210.18)
Track parameters:
	Track ID: 0, TrackParams(pt=608.07, phi=1.06, theta=1.31, charge=-1)
	Track ID: 1, TrackParams(pt=181.03, phi=1.42, theta=1.17, charge=1)
	Track ID: 2, TrackParams(pt=772.37, phi=2.78, theta=2.12, charge=1)
	Track ID: 3, TrackParams(pt=559.30, phi=4.24, theta=1.67, charge=1)
	Track ID: 4, TrackParams(pt=378.67, phi=3.50, theta=2.53, charge=-1)
	Track ID: 5, TrackParams(pt=748.76, phi=1.40, theta=1.81, charge=1)
	Track ID: 6, TrackParams(pt=716.04, phi=6.24, theta=2.92, charge=1)


Event:
Shape of real hits: (253, 3)
Shape of momentums: (253, 3)
Shape of fake hits: (1963, 3)
Fraction of fakes: 0.89
Fraction of missing hits: 0.02
Number of unique tracks: 8
Vertex: Vertex(x=5.60, y=-2.88, z=-92.34)
Track parameters:
	Track ID: 0, TrackParams(pt=588.67, 

In [11]:
draw_event(
    hits=event.hits,
    fakes=event.fakes,
    vertex=event.vertex.numpy,
    labels=event.track_ids,
)

In [7]:
event.track_params

{0: TrackParams(phi=0.6862492958476221, theta=0.5909471191752074, pt=588.6737620151823, charge=1),
 1: TrackParams(phi=5.200627432706265, theta=1.2850740993072944, pt=735.4936826628997, charge=-1),
 2: TrackParams(phi=5.3198299802180635, theta=1.1442118174531155, pt=113.50356070305432, charge=-1),
 3: TrackParams(phi=1.2143013461861, theta=0.539599418237273, pt=330.1356729300098, charge=1),
 4: TrackParams(phi=3.5814472511586612, theta=2.371288116543968, pt=567.1068039403157, charge=1),
 5: TrackParams(phi=6.165089803275703, theta=0.8423417787281019, pt=619.5214484788021, charge=-1),
 6: TrackParams(phi=5.2309446504105175, theta=1.1140020632828307, pt=705.8201459361027, charge=-1),
 7: TrackParams(phi=0.01586268849980221, theta=2.926937431560321, pt=170.8592372658024, charge=1)}

## Example of reproducing tracks by their parameters

In [8]:
track_params = event.track_params
vertex = event.vertex
magnetic_field = event_gen.magnetic_field
z_coord_range = event_gen.z_coord_range
radii = np.linspace(
    event_gen.r_coord_range[0], 
    event_gen.r_coord_range[1],
    event_gen.n_stations
) # mm

hits = []
labels = []

for track in track_params:
    for r in radii:
        hit, _ = SPDEventGenerator.generate_hit_by_params(
            track_params=track_params[track],
            vertex=vertex,
            Rc=r,
            #magnetic_field=magnetic_field
        )

        if (hit.x, hit.y, hit.z) == (0, 0, 0):
            continue

        if not z_coord_range[0] <= hit.z <= z_coord_range[1]:
            continue
        
        hits.append(hit.numpy)
        labels.append(track)
    
hits = np.vstack(hits, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

In [12]:
draw_event(
    hits=hits,
    fakes=None,
    vertex=vertex.numpy,
    labels=labels,
)

## Visualize normalized hits

In [10]:
from src.normalization import ConstraintsNormalizer

normalizer = ConstraintsNormalizer()

draw_event(
    hits=normalizer(event.hits),
    fakes=normalizer(event.fakes),
    vertex=normalizer(np.expand_dims(event.vertex.numpy, 0)).squeeze(0),
    labels=event.track_ids,
    x_coord_range=[-1, 1],
    y_coord_range=[-1, 1],
    z_coord_range=[-1, 1],
)